### Import relevant packages

In [1]:
import time 
import numpy as np 
import pandas as pd

%matplotlib inline
from IPython import display
import matplotlib.colors as colors
import matplotlib.pyplot as plt

### PyTorch packages
import torch
import torch.nn as nn
from torch.autograd import Variable

### Custom functions
def relu(x):
    return torch.max(torch.zeros(1,1),x) # Assuming x is already in torch format

def relu_numpy(x):
    return np.max(np.zeros(1,1),x) # Assuming x is in numpy format

## Deep Learning model with Discriminative Center Loss (DL + DCL)

Coded for the specific case of a 2-layer network (one middle hidden layer and one output layer), for a 2-class (binary) problem. Can be generalized to any number of layers and any number of classes by adjusting the relevant hyperparameters as necessary.

Feature extraction code can be found after the DCLDL model.

In [ ]:
lamb = 1E-1 # Center loss lambda value

num_neurons_array = np.array([32]) # Middle hidden layer
m = num_neurons_array.shape[0] + 1

num_epoch = 1000
num_experiments = 100

d = X.shape[1]

X_train_all = X
y_train_all = y

X_ANN_train = torch.tensor(X_train_all.astype('float32'))

C = np.unique(y_train_all).shape[0] # Number of classes

y_ANN_train = np.eye(C)[y_train_all.astype(int)] # Convert categorical to one-hot-encoding
y_ANN_train = torch.tensor(y_ANN_train.astype('float32'))

## Network loss function w.r.t. parameters:
# Define loss w.r.t. parameters:
def weight_loss_criterion(model,input,target):
    # Grab parameters
    p_dict = {}
    ii = 0
    for p in model.parameters():
        p_dict[ii] = p
        ii += 1

    # Grab parameter norms
    n_params = len(p_dict)
    NN = input.shape[0]

    # Forward prop:
    ZL = {}
    ZL[0] = input
    for ii in range(1,m+1):
        this_W = p_dict[2*ii-2].T
        this_b = p_dict[2*ii-1].reshape(-1,1).T
        ones_matrix = torch.ones((NN,1))
        ZL[ii] = relu( torch.mm(ZL[ii-1],this_W) + torch.mm(ones_matrix,this_b) )
        
    # Backprop:
    temp_softmax = torch.nn.Softmax(dim=1)
    Z_Class_0 = ZL[m][y_train_all==0]
    Z_Class_1 = ZL[m][y_train_all==1]
    
    Z_Center_0 = torch.mean(ZL[m][y_train_all==0],axis=0)
    Z_Center_1 = torch.mean(ZL[m][y_train_all==1],axis=0)
    
    Center_loss = lamb*0.5*( torch.norm((Z_Class_0 - Z_Center_0),2) + torch.norm((Z_Class_1 - Z_Center_1),2) )
    
    loss = torch.norm( ( target - temp_softmax(ZL[m]) ) , 2 ) + Center_loss
    
    return loss, ZL, p_dict

top_Class0_Positive_dict = {}
top_Class0_Negative_dict = {}
top_Class1_Positive_dict = {}
top_Class1_Negative_dict = {}

W_Class0_Positive_dict = {}
W_Class0_Negative_dict = {}
W_Class1_Positive_dict = {}
W_Class1_Negative_dict = {}

### START OF RUNS ###
for run_num in range(0,num_experiments):
    
    p_array = [] # Parameters
    Z_array = [] # Hidden layer values
    loss_array = [] # Total loss

    last_epoch_loss = 1
    time_start = time.time()
    while last_epoch_loss > 0.1:

        ## Initialize Neural Network Architecture
        model = torch.nn.Sequential(
        torch.nn.Linear(d,num_neurons_array[0], bias=True),
        torch.nn.ReLU(),
        torch.nn.Linear(num_neurons_array[0],C, bias=True),
        torch.nn.ReLU(),
        torch.nn.Softmax(dim=1)
        )

        optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay = 0.1)
        loss_list = []

        for epoch in range(0,num_epoch):

            input = Variable(X_ANN_train)
            target = Variable(y_ANN_train)

            # Forward prop
            out = model(input)
            loss, ZL, p_dict = weight_loss_criterion(model,input,target)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch == 0:
                first_epoch_loss = loss.data.item()

            this_loss = loss.data.item()
            loss_list.append(loss.data.item())

            ## Early-stopping criteria 
            earliest_stopping_epoch = 400

            if epoch >= earliest_stopping_epoch and loss.data.item() > 1:
                break

        last_epoch_loss = loss.data.item()
        print("Starting loss of {:.4f} and ending loss of {:.4f}, on epoch {}.".format(first_epoch_loss,last_epoch_loss,epoch))

    print("Run {} now finished, with starting loss of {:.4f} and ending loss of {:.4f}, on epoch {}.".format(run_num,first_epoch_loss,last_epoch_loss,epoch))
    time_end = time.time()
    print("Total elapsed time: {:.1f} sec.".format(time_end-time_start))

    Z_array.append(ZL) # Hidden layer values
    p_array.append(p_dict)

    X_train_all_this = X_train_all

    #### LATENT SPACE PLOTS ####

    ### Grab optimal DL model parameters
    ## Hidden Layer 1: k_1 = 100
    W_1 = p_array[0][0].detach().numpy().T
    b_1 = p_array[0][1].detach().numpy()

    ## Hidden Layer 2 (Pre-Softmax): k_2 = C = 2
    W_2 = p_array[0][2].detach().numpy().T
    b_2 = p_array[0][3].detach().numpy()

    ### Calculate all sample values in Hidden Layers 1 and 2:
    B_1_train = np.ones((N_train,num_neurons_array[0]))
    B_2_train = np.ones((N_train,C))

    for i in range(0,N_train):
        B_1_train[i,:] = b_1.reshape(1,-1)
        B_2_train[i,:] = b_2.reshape(1,-1)

    ## Training samples
    Z_train_HL1 = np.matmul(X_train_all_this,W_1) + B_1_train
    Z_train_HL1 = relu(torch.tensor(Z_train_HL1.astype('float32'))).detach().numpy()

    Z_train_HL2 = np.matmul(Z_train_HL1,W_2) + B_2_train
    Z_train_HL2 = relu(torch.tensor(Z_train_HL2.astype('float32'))).detach().numpy()

    Z_train_HL2_0 = Z_train_HL2[y_train_all==0] # Class 0, ACA+
    Z_train_HL2_1 = Z_train_HL2[y_train_all==1] # Class 1, SCL70+

    ## Latent Space Plots
    dim_1 = 0
    dim_2 = 1

    %matplotlib inline

    plt.figure(figsize=(6,6))
    plt.title("Run {}".format(run_num))
    plt.scatter(Z_train_HL2_0[:,dim_1], Z_train_HL2_0[:,dim_2], marker='o', s=60, color='blue', label = "ACA+ (Class 0)")
    plt.scatter(Z_train_HL2_1[:,dim_1], Z_train_HL2_1[:,dim_2], marker='x', s=60, color='red', label = "SCL70+ (Class 1)")
    plt.xlabel("$Z_1$")
    plt.ylabel("$Z_2$")

    plt.legend(bbox_to_anchor=(2,1))
    plt.show()

    #### FEATURE EXTRACTION (ENSEMBLE APPROACH) ####
    #### Second hidden layer weight traceback
    W_HL2 = p_array[0][2].T.detach().numpy()
    W_HL2_0 = W_HL2[:,0]
    W_HL2_1 = W_HL2[:,1]

    print("Hidden Layer 2 -- Class 0 -- Total number of Positive Weights: {}".format(np.sum(W_HL2_0 > 0)))
    print("Hidden Layer 2 -- Class 0 -- Total number of Negative Weights: {}".format(np.sum(W_HL2_0 < 0)))
    print("Hidden Layer 2 -- Class 1 -- Total number of Positive Weights: {}".format(np.sum(W_HL2_1 > 0)))
    print("Hidden Layer 2 -- Class 1 -- Total number of Negative Weights: {}".format(np.sum(W_HL2_1 < 0)))
    print("")

    m_top_HL2 = 4 # Number of top weights to consider during traceback

    ## Find most positively-correlated weights
    # Z_0 - Pertaining to Class 0
    top_HL2_Class0_Positive = (-W_HL2_0).argsort()[0:m_top_HL2]
    print("Hidden Layer 2 -- Class 0 Most Positive Weights: {}".format(W_HL2_0[top_HL2_Class0_Positive]))
    print("")

    # Z_1 - Pertaining to Class 1
    top_HL2_Class1_Positive = (-W_HL2_1).argsort()[0:m_top_HL2]
    print("Hidden Layer 2 -- Class 1 Most Positive Weights: {}".format(W_HL2_1[top_HL2_Class1_Positive]))
    print("")

    ## Find most negatively-correlated weights
    # Z_0 - Pertaining to Class 0
    top_HL2_Class0_Negative = (W_HL2_0).argsort()[0:m_top_HL2]
    print("Hidden Layer 2 -- Class 0 Most Negative Weights: {}".format(W_HL2_0[top_HL2_Class0_Negative]))
    print("")

    # Z_1 - Pertaining to Class 1
    top_HL2_Class1_Negative = (W_HL2_1).argsort()[0:m_top_HL2]
    print("Hidden Layer 2 -- Class 1 Most Negative Weights: {}".format(W_HL2_1[top_HL2_Class1_Negative]))
    print("")
    print("---------------------------------------------------------------------------------------------------------")

    #### First hidden layer weight traceback
    W_HL1 = p_array[0][0].T.detach().numpy()
    m_top_HL1 = 10 # Number of top weights to consider during traceback

    # Weights most positively-correlated to Class 0
    top_Class0_Positive_pospos = (-W_HL1[:,top_HL2_Class0_Positive]).argsort(axis=0)[0:m_top_HL1]
    top_Class0_Positive_negneg = (W_HL1[:,top_HL2_Class0_Negative]).argsort(axis=0)[0:m_top_HL1]
    top_Class0_Positive = np.concatenate((top_Class0_Positive_pospos,top_Class0_Positive_negneg),axis=1)

    # Weights most positively-correlated to Class 1
    top_Class1_Positive_pospos = (-W_HL1[:,top_HL2_Class1_Positive]).argsort(axis=0)[0:m_top_HL1]
    top_Class1_Positive_negneg = (W_HL1[:,top_HL2_Class1_Negative]).argsort(axis=0)[0:m_top_HL1]
    top_Class1_Positive = np.concatenate((top_Class1_Positive_pospos,top_Class1_Positive_negneg),axis=1)

    # Weights most negatively-correlated to Class 0
    top_Class0_Negative_posneg = (-W_HL1[:,top_HL2_Class0_Negative]).argsort(axis=0)[0:m_top_HL1]
    top_Class0_Negative_negpos = (W_HL1[:,top_HL2_Class0_Positive]).argsort(axis=0)[0:m_top_HL1]
    top_Class0_Negative = np.concatenate((top_Class0_Negative_posneg,top_Class0_Negative_negpos),axis=1)

    # Weights most negatively-correlated to Class 1
    top_Class1_Negative_posneg = (-W_HL1[:,top_HL2_Class1_Negative]).argsort(axis=0)[0:m_top_HL1]
    top_Class1_Negative_negpos = (W_HL1[:,top_HL2_Class1_Positive]).argsort(axis=0)[0:m_top_HL1]
    top_Class1_Negative = np.concatenate((top_Class1_Negative_posneg,top_Class1_Negative_negpos),axis=1)

    # Compile results
    W_Class0_Positive = np.zeros((m_top_HL1,m_top_HL2*2))
    W_Class1_Positive = np.zeros((m_top_HL1,m_top_HL2*2))
    W_Class0_Negative = np.zeros((m_top_HL1,m_top_HL2*2))
    W_Class1_Negative = np.zeros((m_top_HL1,m_top_HL2*2))

    for j in range(0,m_top_HL2):
        W_Class0_Positive[:,j] = W_HL1[:,top_HL2_Class0_Positive][:,j][top_Class0_Positive_pospos[:,j]]
        W_Class0_Positive[:,j+m_top_HL2] = W_HL1[:,top_HL2_Class0_Negative][:,j][top_Class0_Positive_negneg[:,j]]

        W_Class1_Positive[:,j] = W_HL1[:,top_HL2_Class1_Positive][:,j][top_Class1_Positive_pospos[:,j]]
        W_Class1_Positive[:,j+m_top_HL2] = W_HL1[:,top_HL2_Class1_Negative][:,j][top_Class1_Positive_negneg[:,j]]

        W_Class0_Negative[:,j] = W_HL1[:,top_HL2_Class0_Negative][:,j][top_Class0_Negative_posneg[:,j]]
        W_Class0_Negative[:,j+m_top_HL2] = W_HL1[:,top_HL2_Class0_Positive][:,j][top_Class0_Negative_negpos[:,j]]

        W_Class1_Negative[:,j] = W_HL1[:,top_HL2_Class1_Negative][:,j][top_Class1_Negative_posneg[:,j]]
        W_Class1_Negative[:,j+m_top_HL2] = W_HL1[:,top_HL2_Class1_Positive][:,j][top_Class1_Negative_negpos[:,j]]

    top_Class0_Positive_dict[run_num] = top_Class0_Positive
    top_Class0_Negative_dict[run_num] = top_Class0_Negative
    top_Class1_Positive_dict[run_num] = top_Class1_Positive
    top_Class1_Negative_dict[run_num] = top_Class1_Negative

    W_Class0_Positive_dict[run_num] = W_Class0_Positive
    W_Class0_Negative_dict[run_num] = W_Class0_Negative
    W_Class1_Positive_dict[run_num] = W_Class1_Positive
    W_Class1_Negative_dict[run_num] = W_Class1_Negative

In [ ]:
#### Consolidate results ####
def ensemblefy_DLFE(indices_dict,importances_dict,m_top_HL1,m_top_HL2):

    final_DLFE_importances = np.zeros((m_top_HL1,2*m_top_HL2))
    final_DLFE_indices = np.zeros((m_top_HL1,2*m_top_HL2))

    for j in range(0,2*m_top_HL2):
        these_DLFE_importances = []
        these_DLFE_indices = []

        for i in range(0,num_experiments):
            these_DLFE_indices.append(indices_dict[i][:,j])
            these_DLFE_importances.append(importances_dict[i][:,j])

        these_DLFE_importances = np.abs(np.array(these_DLFE_importances))
        these_DLFE_indices = np.array(these_DLFE_indices)

        unique_DLFE_indices = np.unique(these_DLFE_indices)
        unique_DLFE_importances = np.zeros((unique_DLFE_indices.shape[0],))

        for ii in range(0,unique_DLFE_indices.shape[0]):
            unique_DLFE_importances[ii] = np.sum(these_DLFE_importances[these_DLFE_indices == unique_DLFE_indices[ii]])

        final_DLFE_importances[:,j] = -np.sort(-unique_DLFE_importances)[0:m_top_HL1]
        eee = unique_DLFE_indices[np.argsort(-unique_DLFE_importances)[0:m_top_HL1]]
        final_DLFE_indices[:,j] = eee.astype(int)

    return final_DLFE_indices, final_DLFE_importances

## Class 0 Positively-correlated
[Class0_Positive_Final_Indices, Class0_Positive_Final_Importances] = ensemblefy_DLFE(top_Class0_Positive_dict,W_Class0_Positive_dict,m_top_HL1,m_top_HL2)
Class0_Positive_Final_Importances = np.abs(Class0_Positive_Final_Importances)
Class0_Positive_Final_Indices = Class0_Positive_Final_Indices.astype(int)

## Class 0 Negative-correlated
[Class0_Negative_Final_Indices, Class0_Negative_Final_Importances] = ensemblefy_DLFE(top_Class0_Negative_dict,W_Class0_Negative_dict,m_top_HL1,m_top_HL2)
Class0_Negative_Final_Importances = -np.abs(Class0_Negative_Final_Importances)
Class0_Negative_Final_Indices = Class0_Negative_Final_Indices.astype(int)

## Class 1 Positively-correlated
[Class1_Positive_Final_Indices, Class1_Positive_Final_Importances] = ensemblefy_DLFE(top_Class1_Positive_dict,W_Class1_Positive_dict,m_top_HL1,m_top_HL2)
Class1_Positive_Final_Importances = np.abs(Class1_Positive_Final_Importances)
Class1_Positive_Final_Indices = Class1_Positive_Final_Indices.astype(int)

## Class 1 Negative-correlated
[Class1_Negative_Final_Indices, Class1_Negative_Final_Importances] = ensemblefy_DLFE(top_Class1_Negative_dict,W_Class1_Negative_dict,m_top_HL1,m_top_HL2)
Class1_Negative_Final_Importances = -np.abs(Class1_Negative_Final_Importances)
Class1_Negative_Final_Indices = Class1_Negative_Final_Indices.astype(int)

## Deep Learning model without Discriminative Center Loss (DL only)

Coded for the same specific case above, without DCL component or final feature extraction.

In [ ]:
X = X_scaled
y = y_labels


num_neurons_array = np.array([32])
m = num_neurons_array.shape[0] + 1

num_epoch = 1000
num_experiments = 100

d = X.shape[1]

X_train_all = X
y_train_all = y

X_ANN_train = torch.tensor(X_train_all.astype('float32'))

C = np.unique(y_train_all).shape[0] # Number of classes

y_ANN_train = np.eye(C)[y_train_all.astype(int)] # Convert categorical to one-hot-encoding
y_ANN_train = torch.tensor(y_ANN_train.astype('float32'))

FE_train_acc_array = []
FE_test_acc_array = []

y_train_dict = {}
y_train_hat_dict = {}
y_test_hat_dict = {}

p_array = [] # Parameters
Z_array = [] # Hidden layer values
loss_array = [] # Total loss

## Neural Network Architecture
model = torch.nn.Sequential(
torch.nn.Linear(d,num_neurons_array[0], bias=True),
torch.nn.ReLU(),
torch.nn.Linear(num_neurons_array[0],C, bias=True),
torch.nn.ReLU(),
torch.nn.Softmax(dim=1)
)

## Network loss function w.r.t. parameters:
# Define loss w.r.t. parameters:
def parameter_grab(model,input,target):
    # Grab parameters
    p_dict = {}
    ii = 0
    for p in model.parameters():
        p_dict[ii] = p
        ii += 1

    # Grab parameter norms
    n_params = len(p_dict)
    p_norm_dict = {}
    for ii in range(0,n_params):
        p_norm_dict[ii] = torch.norm(p_dict[ii],2)

    NN = input.shape[0]

    # Forward prop:
    ZL = {}
    ZL[0] = input
    for ii in range(1,m+1):
        this_W = p_dict[2*ii-2].T
        this_b = p_dict[2*ii-1].reshape(-1,1).T
        ones_matrix = torch.ones((NN,1))
        ZL[ii] = relu( torch.mm(ZL[ii-1],this_W) + torch.mm(ones_matrix,this_b) )

    return ZL, p_dict, p_norm_dict

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay = 0.01)

loss_list = []

for epoch in range(0,num_epoch):
            
    input = Variable(X_ANN_train)
    target = Variable(y_ANN_train)

    # Forward prop
    out = model(input)
    loss = loss_function(out, target)
    ZL, p_dict, p_norm_dict = parameter_grab(model,input,target)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print results per epoch
    print('Epoch[{}/{}], loss: {:.6f}'
          .format(epoch + 1, num_epoch, loss.data.item()))
    this_loss = loss.data.item()
    loss_list.append(loss.data.item())

Z_array.append(ZL) # Hidden layer values
print("This randomly-initialized ANN has finished training, with a final loss of {:.3f}.".format(this_loss))
print("")

### Model accuracy evaluation (overall and class-specific)

In [ ]:
#### EVALUATE OVERALL MODEL ACCURACIES ####
y_ANN_train_hat_np = model(X_ANN_train).data.numpy()
y_ANN_train_hat = np.argmax(y_ANN_train_hat_np,axis=1)
ANN_train_acc = np.sum(y_ANN_train_hat == y_train_all)/y_ANN_train_hat.shape[0]*100
print("Overall Training Accuracy: {:.1f}%".format(ANN_train_acc))

#### EVALUATE INDIVIDUAL ACCURACIES ####
for c in range(0,C):
    y_class_train_hat = np.argmax(model(X_ANN_train[y_train_all == c]).data.numpy(),axis=1)
    y_class_train = y_train_all[y_train_all == c]
    class_train_acc = np.sum(y_class_train_hat == y_class_train)/y_class_train_hat.shape[0]*100
    print("Class {} Testing Accuracy: {:.1f}%".format(c,class_train_acc))